In [2]:
!pip install tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 3.5 MB/s eta 0:00:00


In [7]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from torch.utils.data import TensorDataset
from tqdm import tqdm

In [4]:
# hyper-parameters
timestep = 1 # time window size
batch_size = 16
input_dim = 14
hidden_dim = 64
output_dim = 1
num_layers = 3
epochs = 10
best_loss = 0
model_name = 'gru'
save_path = './{}.pth'.format(model_name)

In [9]:
# 1. load data
df = pd.read_csv('./data/jena_climate_2009_2016.csv',index_col=0)
df.head()

,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),rho (g/m**3),wv (m/s),max. wv (m/s),wd (deg)
Date Time,,,,,,,,,,,,,,
01.01.2009 00:10:00,996.52,-8.02,265.40,-8.90,93.3,3.33,3.11,0.22,1.94,3.12,1307.75,1.03,1.75,152.3
01.01.2009 00:20:00,996.57,-8.41,265.01,-9.28,93.4,3.23,3.02,0.21,1.89,3.03,1309.80,0.72,1.50,136.1
01.01.2009 00:30:00,996.53,-8.51,264.91,-9.31,93.9,3.21,3.01,0.20,1.88,3.02,1310.24,0.19,0.63,171.6
01.01.2009 00:40:00,996.51,-8.31,265.12,-9.07,94.2,3.26,3.07,0.19,1.92,3.08,1309.19,0.34,0.50,198.0
01.01.2009 00:50:00,996.51,-8.27,265.15,-9.04,94.1,3.27,3.08,0.19,1.92,3.09,1309.00,0.32,0.63,214.3


In [10]:
# 2. standardize data
scaler = StandardScaler()
scaler_model = StandardScaler()
data = scaler_model.fit_transform(np.array(df))
scaler.fit_transform(np.array(df['T (degC)']).reshape(-1,1))   # T (degC) is the target

array([[-2.0740129 ],
       [-2.12031274],
       [-2.1321845 ],
       ...,
       [-1.49704566],
       [-1.62407343],
       [-1.69411678]])

In [14]:
# 3. generate training data, like 1234 -> 5, 2345 -> 6, ...
def split_data(data, timestep, input_dim):
    dataX = []
    dataY = []

    for index in range(len(data) - timestep):
        dataX.append(data[index:index+timestep])
        dataY.append(data[index+timestep][1])
    
    dataX = np.array(dataX)
    dataY = np.array(dataY)

    # get training data size
    train_size = int(np.round(0.8*dataX.shape[0]))
    # split data into training and testing
    x_train = dataX[:train_size,:].reshape(-1,timestep,input_dim)
    y_train = dataY[:train_size]

    x_test = dataX[train_size:,:].reshape(-1,timestep,input_dim)
    y_test = dataY[train_size:]

    return [x_train, y_train, x_test, y_test]

In [17]:
# 4. getting training data
x_train, y_train, x_test, y_test = split_data(data, timestep, input_dim)

# 5. turn to TensorDataset
x_train_tensor = torch.from_numpy(x_train).to(torch.float32)
y_train_tensor = torch.from_numpy(y_train).to(torch.float32)
x_test_tensor = torch.from_numpy(x_test).to(torch.float32)
y_test_tensor = torch.from_numpy(y_test).to(torch.float32)

# 6. generate dataloader
train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
test_dataset = TensorDataset(x_test_tensor, y_test_tensor)

train_loader = torch.utils.data.DataLoader(
    dataset = train_dataset,
    batch_size = batch_size,
    shuffle = True,
)

test_loader = torch.utils.data.DataLoader(
    dataset = test_dataset,
    batch_size = batch_size,
    shuffle = False,
)

In [18]:
# 7. define GRU model
class GRU(nn.Module):
    def __init__(self,input_dim,hidden_dim,num_layers,output_dim):
        super(GRU,self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers # LSTM layers
        self.gru = nn.GRU(input_dim,hidden_dim,num_layers,batch_first=True) # batch_first means (batch, seq, feature)
        self.fc = nn.Linear(hidden_dim,output_dim)

    def forward(self,x):
        output, h_n = self.gru(x) # output size: (batch, seq, hidden_dim)
        batch_size, timestep, hidden_dim = output.shape

        output = output.reshape(-1,hidden_dim)
        output = self.fc(output)
        output = output.reshape(timestep, batch_size,-1) # (timestep, batch, output_dim)

        return output[-1] # (batch, output_dim)

In [19]:
model = GRU(input_dim,hidden_dim,num_layers,output_dim)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.01)

In [20]:
# 8. training
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    train_bar = tqdm(train_loader) # show progress bar
    for data in train_bar:
        x_train, y_train = data # get data
        optimizer.zero_grad() # clear gradient
        y_train_pred = model(x_train)
        loss = loss_function(y_train_pred,y_train.reshape(-1,1))
        loss.backward() # back propagation
        optimizer.step() # update parameters

        running_loss += loss.item()
        train_bar.desc = "train epoch[{}/{}] loss:{:.3f}".format(epoch+1,epochs,loss)

    # model evaluation
    model.eval()
    test_loss = 0.0
    with torch.no_grad():
        test_bar = tqdm(test_loader)
        for data in test_bar:
            x_test, y_test = data
            y_test_pred = model(x_test)
            test_loss = loss_function(y_test_pred,y_test.reshape(-1,1))

print('Finish Training')

train epoch[10/10] loss:0.000: 100%|██████████| 21028/21028 [01:01<00:00, 340.41it/s]


In [ ]:
# plot results
plt.figure(figsize=(12,8))
plt.plot(scaler.inverse_transform((model(x_train_tensor).detach().numpy()).reshape(-1,1)),"b")
plt.plot(scaler.inverse_transform(y_train_tensor.detach().numpy().reshape(-1,1)),"r")
plt.legend()
plt.show()

y_test_pred = model(x_test_tensor)
plt.figure(figsize=(12,8))
plt.plot(scaler.inverse_transform(y_test_pred.detach().numpy()),"b")
plt.plot(scaler.inverse_transform(y_test_tensor.detach().numpy().reshape(-1,1)),"r")
plt.legend()
plt.show()